<a href="https://colab.research.google.com/github/gin614pham/Server_wav2vec2_vi/blob/main/Wav2vec_vi_no2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_checkpoint = "facebook/wav2vec2-large-xlsr-53"
batch_size = 16

In [2]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("multi_lingual_speech_recognition_notebook", framework="pytorch")

In [6]:
from datasets import load_dataset

In [51]:
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "vi", split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_11_0", "vi", split="test")

In [52]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [53]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [54]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence,variant
0,Bước từng nhịp một hai như đang tiến vào ngôi làng,
1,Sau đó Phương chị về phía trước miệng không ngừng hét,
2,Nhìn đồng hồ lúc này cũng đã là mười giờ tối,
3,Vì quán nằm ở trong hẻm,
4,Mở chiếc cửa sổ căn nhà của nó ra nhìn,
5,Chỉ anh thôi ai để ý đâu mà,
6,Tiếng mõ tụng kinh vẫn còn vang lên tới điện chính,
7,Không biết Sao con Thảo nó lại không có báo mộng cho chúng tôi,
8,Khi được vuông tròn,
9,Chăm sóc kuman Càng tốt càng hiệu quả,


In [55]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [56]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [57]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,variant
0,lạ cái lúc về,
1,ôi con ơi con mở mắt rồi sao,
2,lên núi đặt lờ,
3,cô vừa bảo là có con nhỏ bạn thân bị chết thảm đó sao,
4,nên bị ngã ngửa lên giường,
5,đừng có lo,
6,mẹ không ngủ được,
7,em giật mình nói,
8,tại sao lại bắt cháu,
9,buồn thế chứ,


In [58]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [59]:
vocab_train = common_voice_train.map(
  extract_all_chars, batched=True,
  batch_size=-1, keep_in_memory=True,
  remove_columns=common_voice_train.column_names
)
vocab_test = common_voice_test.map(
  extract_all_chars, batched=True,
  batch_size=-1, keep_in_memory=True,
  remove_columns=common_voice_test.column_names
)

Map:   0%|          | 0/2939 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [60]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [61]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'r': 0,
 'ề': 1,
 "'": 2,
 'o': 3,
 'f': 4,
 'ễ': 5,
 'p': 6,
 't': 7,
 'ã': 8,
 'w': 9,
 'ô': 10,
 'v': 11,
 'ẫ': 12,
 'á': 13,
 'd': 14,
 'ĩ': 15,
 'u': 16,
 ' ': 17,
 'ế': 18,
 'ở': 19,
 'ó': 20,
 'c': 21,
 'ữ': 22,
 'ẳ': 23,
 'ê': 24,
 'ẵ': 25,
 'ư': 26,
 'ủ': 27,
 'ự': 28,
 'ỹ': 29,
 'ỗ': 30,
 'ổ': 31,
 's': 32,
 'â': 33,
 'è': 34,
 'ũ': 35,
 'õ': 36,
 'ơ': 37,
 'ả': 38,
 'ặ': 39,
 'ù': 40,
 'í': 41,
 'ẩ': 42,
 'ạ': 43,
 'x': 44,
 'ò': 45,
 'ỡ': 46,
 'ă': 47,
 'ừ': 48,
 'ị': 49,
 'ệ': 50,
 'ý': 51,
 'm': 52,
 'ầ': 53,
 'ỳ': 54,
 'z': 55,
 'a': 56,
 'ẹ': 57,
 'ứ': 58,
 'g': 59,
 'ỏ': 60,
 'ử': 61,
 'à': 62,
 'ậ': 63,
 'h': 64,
 'k': 65,
 'é': 66,
 'ụ': 67,
 'ể': 68,
 'đ': 69,
 'ỵ': 70,
 'ồ': 71,
 'ỉ': 72,
 'y': 73,
 'b': 74,
 'l': 75,
 'i': 76,
 'ì': 77,
 'ố': 78,
 'ộ': 79,
 'ỷ': 80,
 'ấ': 81,
 'ợ': 82,
 'ắ': 83,
 'ẻ': 84,
 'ú': 85,
 'e': 86,
 'ằ': 87,
 'ớ': 88,
 'ờ': 89,
 'ọ': 90,
 'ẽ': 91,
 'n': 92,
 'q': 93,
 'j': 94}

In [62]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [63]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

97

In [64]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [65]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None

In [66]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  "./",
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.


In [67]:
model_checkpoint_name = model_checkpoint.split("/")[-1]
repo_name = f"{model_checkpoint_name}-vietnamese-finetuned-demo-colab"

In [68]:
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-vietnamese-finetuned-demo-colab/commit/4ac2f1e60b0deae52d1296e64069a50a4fb8467a', commit_message='Upload tokenizer', commit_description='', oid='4ac2f1e60b0deae52d1296e64069a50a4fb8467a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-vietnamese-finetuned-demo-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='ginpham614/wav2vec2-large-xlsr-53-vietnamese-finetuned-demo-colab'), pr_revision=None, pr_num=None)

In [69]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/988a1303a5bdc3cbd1fbdc27b2aeeb0c081ca80a35035ee932f592ffd443f607/vi_train_0/common_voice_vi_23901117.mp3'

In [70]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/988a1303a5bdc3cbd1fbdc27b2aeeb0c081ca80a35035ee932f592ffd443f607/vi_train_0/common_voice_vi_23901117.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.90132641e-06, 7.18296724e-06, 1.22913943e-05]),
 'sampling_rate': 48000}

In [71]:
from datasets import Audio


In [72]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [73]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/988a1303a5bdc3cbd1fbdc27b2aeeb0c081ca80a35035ee932f592ffd443f607/vi_train_0/common_voice_vi_23901117.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.79643739e-05, -1.54107620e-05,  3.32880700e-06]),
 'sampling_rate': 16000}

In [74]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

ngôi nhà chưa xảy ra chuyện gì 


In [75]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: chứ không phải là hà ni 
Input array shape: (44928,)
Sampling rate: 16000


In [76]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

In [77]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [78]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [79]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

In [80]:
max_input_length_in_sec = 10.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/2939 [00:00<?, ? examples/s]

In [81]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [82]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [83]:
!pip install evaluate



In [84]:
import evaluate

In [85]:
  wer_metric = evaluate.load("wer")

In [86]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [87]:
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained(
    model_checkpoint,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
if hasattr(model, "freeze_feature_extractor"):
  model.freeze_feature_extractor()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2176: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [89]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=batch_size,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [90]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

<ipython-input-90-ec2c74f250fd>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [91]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,8.910000,3.494872,1.000000
800,2.915700,1.298718,0.920478
1200,0.534600,1.029718,0.677196
1600,0.273500,0.994845,0.620270
2000,0.187200,1.029306,0.586377
2400,0.141500,1.058082,0.573983


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in 

TrainOutput(global_step=2760, training_loss=1.8948950698410256, metrics={'train_runtime': 8270.0974, 'train_samples_per_second': 10.661, 'train_steps_per_second': 0.334, 'total_flos': 1.177342743567614e+19, 'train_loss': 1.8948950698410256, 'epoch': 30.0})

In [92]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-vietnamese-finetuned-demo-colab/commit/ca383b44226774850ceab0607a78ae2f1412571b', commit_message='End of training', commit_description='', oid='ca383b44226774850ceab0607a78ae2f1412571b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ginpham614/wav2vec2-large-xlsr-53-vietnamese-finetuned-demo-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='ginpham614/wav2vec2-large-xlsr-53-vietnamese-finetuned-demo-colab'), pr_revision=None, pr_num=None)